In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Get unique launch sites from the dataframe
launch_sites = spacex_df['Launch Site'].unique()

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # TASK 1: Add a dropdown list to enable Launch Site selection
    dcc.Dropdown(id='site-dropdown',
                 options=[{'label': 'All Sites', 'value': 'ALL'}] +
                         [{'label': site, 'value': site} for site in launch_sites],
                 value='ALL',
                 placeholder="Select a launch site",
                 searchable=True,
                 style={'width': '80%'}
                 ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),

    # TASK 3: Add a slider to select payload range
    dcc.RangeSlider(id='payload-slider',
                    min=min_payload, max=max_payload, step=1000,
                    marks={i: str(i) for i in range(0, int(max_payload) + 1000, 1000)},
                    value=[min_payload, max_payload]),

    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2:
# Add a callback function for site-dropdown as input, success-pie-chart as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(spacex_df, values='Flight Number', names='class', title='Total Success Launches by Site')
    else:
        filtered_data = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(filtered_data, names='class', title=f'Success Launches for site {selected_site}')
    return fig


# TASK 4:
# Add a callback function for site-dropdown and payload-slider as inputs, success-payload-scatter-chart as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_chart(selected_site, payload_range):
    if selected_site == 'ALL':
        filtered_data = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                  (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        title = f'Correlation between Payload and Success for all Sites'
    else:
        filtered_data = spacex_df[(spacex_df['Launch Site'] == selected_site) &
                                  (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                  (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        title = f'Correlation between Payload and Success for site {selected_site}'
    
    fig = px.scatter(filtered_data, x='Payload Mass (kg)', y='class', color='Booster Version Category', 
                     title=title)
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(port=8051,debug=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3216\1463192767.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3216\1463192767.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
